# 2. Companies House API

* Tricky to get the key to work, but eventually managed to figure out how to get what I needed from companies house.

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time, random

In [2]:
GPG_18_19 = pd.read_csv('/Users/gitas/Desktop/GA/Capstone/Gender_Pay_Gap_Data/UK_Gender_Pay_Gap_Data_2018_2019.csv')

In [3]:
GPG_18_19.dropna(subset=['CompanyNumber', 'DiffMeanBonusPercent', 'DiffMedianBonusPercent'], inplace=True)

In [4]:
GPG_18_19.reset_index(drop=True, inplace=True)

In [5]:
GPG_18_19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8105 entries, 0 to 8104
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   EmployerName               8105 non-null   object 
 1   Address                    8105 non-null   object 
 2   CompanyNumber              8105 non-null   object 
 3   SicCodes                   7713 non-null   object 
 4   DiffMeanHourlyPercent      8105 non-null   float64
 5   DiffMedianHourlyPercent    8105 non-null   float64
 6   DiffMeanBonusPercent       8105 non-null   float64
 7   DiffMedianBonusPercent     8105 non-null   float64
 8   MaleBonusPercent           8105 non-null   float64
 9   FemaleBonusPercent         8105 non-null   float64
 10  MaleLowerQuartile          8105 non-null   float64
 11  FemaleLowerQuartile        8105 non-null   float64
 12  MaleLowerMiddleQuartile    8105 non-null   float64
 13  FemaleLowerMiddleQuartile  8105 non-null   float

<font color='red'>

#### Note - for all the below scrapes, since I was limited to 600 requests per 5 minutes, I did these for 600 companies then set a timer for 6 minutes before running for the next 600 etc, each time appending to the dictionary. Once data for all companies were scraped, I converted the dictionary to a dataframe and saved to a csv file. 

</font>

### Step 1: Company Overview - I utilised the Companies House API to obtain basic company information such as the company number, status, creation date, jurisdiction, address and type.

In [6]:
companies_house = {'company_no' : [],
                    'company_status' : [],
                    'creation_date' : [],
                    'jurisdiction' : [],
                    'registered_address_country' : [],
                    'company_type' : []}

In [7]:
import time
from pprint import pprint
from requests.auth import HTTPBasicAuth
import base64
import requests
apiKey = 'F11LgkEtJd6NvuWGCs7n88nlbaXp3cC5JjJ85sP6:'
list_companies_dict = []
    
for company_number in GPG_18_19['CompanyNumber'][100:110]:
    url1 = 'https://api.companieshouse.gov.uk/company/' + company_number
    re = requests.get(url1,auth=HTTPBasicAuth(apiKey,'')).json()
    list_companies_dict.append(re)

In [8]:
for item in list_companies_dict:
    try:
        companies_house['company_no'].append(item['company_number'])
    except:
        companies_house['company_no'].append('None')

    try:
        companies_house['company_status'].append(item['company_status'])
    except:
        companies_house['company_status'].append('None')


    try:
        companies_house['creation_date'].append(item['date_of_creation'])
    except:
        companies_house['creation_date'].append('None')

    try:
        companies_house['jurisdiction'].append(item['jurisdiction'])
    except:
        companies_house['jurisdiction'].append('None')

    try:
        companies_house['registered_address_country'].append(item['registered_office_address']['country'])
    except:
        companies_house['registered_address_country'].append('None')

    try:
        companies_house['company_type'].append(item['type'])
    except:
        companies_house['company_type'].append('None')

companies_house = pd.DataFrame(companies_house, index = companies_house['company_no'])
companies_house
# companies_house.to_csv('companies_house_18_19')

,company_no,company_status,creation_date,jurisdiction,registered_address_country,company_type
03070774,03070774,active,1995-06-21,england-wales,England,ltd
03484784,03484784,administration,1997-12-23,england-wales,None,ltd
03725308,03725308,active,1999-03-03,england-wales,None,ltd
07199239,07199239,active,2010-03-23,england-wales,None,ltd
10248411,10248411,active,2016-06-23,england-wales,England,ltd
02764368,02764368,active,1992-11-13,england-wales,England,ltd
SC137975,SC137975,active,1992-04-27,scotland,Scotland,ltd
02310474,02310474,active,1988-10-28,england-wales,United Kingdom,ltd
07654902,07654902,active,2011-06-01,england-wales,None,private-limited-guarant-nsc-limited-exemption
01636368,01636368,active,1982-05-18,england-wales,None,ltd


### Step 2: Officers - I utilised the Companies House API to obtain a list of all company officers, this was then used to calculate the % female officers for each company

In [9]:
officers = {'company_no' : [],
            'name' : [],
            'date_appointed' : [],
            'date_resigned' : []}

In [10]:
list_companies_dict = []

for company_number in GPG_18_19['CompanyNumber'][100:110]:
    url2 = 'https://api.companieshouse.gov.uk/company/' + company_number + '/officers'
    re = requests.get(url2,auth=HTTPBasicAuth(apiKey,'')).json()
    list_companies_dict.append(re)

In [11]:
for comp in list_companies_dict:
    try:
        for item in comp['items']:
            try:
                officers['company_no'].append(item['links']['self'][9:17])
            except:
                officers['company_no'].append('None')

            try:
                officers['name'].append(item['name'])
            except:
                officers['name'].append('None')

            try:
                officers['date_appointed'].append(item['appointed_on'])
            except:
                officers['date_appointed'].append('None')

            try:
                officers['date_resigned'].append(item['resigned_on'])
            except:
                officers['date_resigned'].append('None')
    except:
        pass

officers = pd.DataFrame(officers, index = officers['company_no'])
officers
# officers.to_csv('officers_18_19_v2')

,company_no,name,date_appointed,date_resigned
03070774,03070774,"GATTY, Glen Michael",2005-06-10,None
03070774,03070774,"BRIMELOW, Richard John",2020-10-30,None
03070774,03070774,"GATTY, Glen Michael",2008-01-01,None
03070774,03070774,"GOOCH, David",2013-12-16,None
03070774,03070774,"PROTHERO, Nigel",2005-06-10,None
...,...,...,...,...
01636368,01636368,"STEVENSON, Nicholas John",2002-04-22,2014-02-17
01636368,01636368,"WEST, Maria Theresa Johanna",None,1992-09-05
01636368,01636368,"WEST, Susan Maureen",1995-02-15,2002-04-22
01636368,01636368,"PROBERT, Dawn",2019-11-15,2020-08-14


### Step 3: Company Location - I utilised the Companies House API to obtain information about the company location. Whilst this was also included in the original government gender pay gap dataset, this information needed to be cleaned and was in a single string. I therefore decided to scrape this information from companies house, only using the sections of the address I was interested in (locality and country) rather than the full address. 

In [12]:
company_location = {'company_no' : [],
                    'locality' : [],
                    'country' : []}

In [14]:
apiKey = 'F11LgkEtJd6NvuWGCs7n88nlbaXp3cC5JjJ85sP6:'
list_companies_dict = []
    
for company_number in GPG_18_19['CompanyNumber'][100:110]:
    url1 = 'https://api.companieshouse.gov.uk/company/' + company_number
    re = requests.get(url1,auth=HTTPBasicAuth(apiKey,'')).json()
    list_companies_dict.append(re)

In [15]:
for item in list_companies_dict:
    try:
        company_location['company_no'].append(item['company_number'])
    except:
        company_location['company_no'].append('None')

    try:
        company_location['locality'].append(item['registered_office_address']['locality'])
    except:
        company_location['locality'].append('None')
    
    try:
        company_location['country'].append(item['registered_office_address']['country'])
    except:
        company_location['country'].append('None')
        
company_location = pd.DataFrame(company_location, index = company_location['company_no'])
company_location
# company_location.to_csv('company_location_18_19')

,company_no,locality,country
03070774,03070774,Dorking,England
03484784,03484784,Bristol,None
03725308,03725308,Willenhall,None
07199239,07199239,Chorley,None
10248411,10248411,Leeds,England
02764368,02764368,Manchester,England
SC137975,SC137975,Edinburgh,Scotland
02310474,02310474,Surrey,United Kingdom
07654902,07654902,Warminster,None
01636368,01636368,Liverpool,None
